In [1]:
import chainer
from util import *

/opt/pyenv/versions/3.6.5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np

file_path='share/datasets/LFW/lfw_attributes.txt'
attribute_dataset = load_attribute_dataset(file_path)
attribute = find_attribute(attribute_dataset, b'Aaron Eckhart', 1)
print(attribute)

[ 1.5683464  -1.8890427   1.7372032  -0.9297287  -1.4717995  -0.19558042
 -0.8356094  -0.35146832 -1.0125334  -0.71959335 -0.6324007   0.46483916
 -0.9735283   1.5651855  -1.2967042  -1.5427188  -0.68467104 -0.8649897
  0.76688576 -0.2189521  -1.6556655  -0.7870439  -0.5996649   0.4585186
  0.18975967  0.8515547  -0.3857204  -0.49771923 -0.16114904 -0.25751433
 -0.08883881  0.4554688  -0.8392114  -0.02294812 -0.92256767 -0.11453859
  1.4612217   1.7584809   0.06889351  1.2678598  -1.1202441   0.91761655
 -1.3079566  -1.5004133   1.0292207   0.83236295 -0.498657    0.251365
 -0.7052813  -0.5157155   0.37423918 -0.1686746  -0.61414325  3.0977027
  1.5238582   0.779278   -0.07145392 -1.2464834  -0.7692835  -0.7255967
 -1.8206103  -2.0729766  -0.96075875  0.3617377   1.1661183  -1.1649163
 -1.1399903  -2.3717458  -1.299932   -0.41468176 -1.1449021   0.6940072
 -0.8266088 ]


In [3]:
import math
import numpy as np
import os

class Config(object):
    celebA_dir = 'share/datasets/CelebA/'
    ms_celeb_dir='share/datasets/MSCeleb1M/raw'
    #pretrainmodel_path='inception_resnet.npz'
    #pretrainmodel_path='result/Classification_40000.npz'
    pretrainmodel_path='msceleb_pretrain2/Classification_510000.npz'
    lr=0.01
    gpu=5

class Args(object):
    display_interval=10
    evaluation_interval=100
    evaluation_interval2=1000
    lr=0.01
    gpu=5
    batchsize=30
    max_iter= 1661926/batchsize*30
    epoch_iter= 1661926/batchsize
    #out='d2ae_small'
    #out='d2ae_msceleba'
    out='d2ae_autoencoder'
    display_interval=10
    snapshot_interval=100000

In [4]:
from chainer import serializers
from encoder import InceptionResNetV2
config=Config()
encoder=InceptionResNetV2().to_gpu(config.gpu)
#serializers.load_npz(config.pretrainmodel_path, encoder)


In [5]:

import numpy as np

import chainer
import chainer.functions as F
from chainer import Variable
import cupy

class Updater(chainer.training.StandardUpdater):
    def __init__(self, *args, **kwargs):
        self.d2ae, self.ip = kwargs.pop('models')
        self.config = kwargs.pop('config')
        self.lamdat=1
        self.lamdap=0.1
        self.lamdax= 1.81e-5
        
        super(Updater, self).__init__(*args, **kwargs)

    def update_core(self):
        d2ae_optimizer = self.get_optimizer('opt_d2ae')
        ip_optimizer = self.get_optimizer('opt_ip')
        xp = self.d2ae.xp

        batch, batch_out, label = self.get_iterator('main').next()
        batchsize = len(batch)
        x = []; x_out=[]
        for i in range(batchsize):
                x.append(np.asarray(batch[i]).astype("f"))
                x_out.append(np.asarray(batch_out[i]).astype("f"))
        with cupy.cuda.Device(self.config.gpu):
            x_in = Variable(xp.asarray(x))
            x_out = Variable(xp.asarray(x_out))
            label = Variable(xp.asarray(label, dtype=np.int32))

        X,X_hat,yt,fp = self.d2ae(x_in)
        #fp=Variable(fp.data)
        yp=self.ip(fp)
        loss_I = F.softmax_cross_entropy(yt,label)
        #print('X',X)
        #print('x',x_out)
        loss_X = F.mean_squared_error(X,x_out)*X.shape[1]*X.shape[2]*X.shape[3]
        loss_Xh = F.mean_squared_error(X_hat,x_out)*X.shape[1]*X.shape[2]*X.shape[3]
        loss_H = F.softmax_cross_entropy(-yp,label)
        loss1 = self.lamdax*(loss_X)
        #loss1 = self.lamdax*(loss_X) 
        #loss1 = self.lamdax*loss_X*X.shape[1]*X.shape[2]*X.shape[3]
        self.d2ae.cleargrads()
        loss1.backward()
        d2ae_optimizer.update()
        chainer.reporter.report({'loss_I': loss_I})        
        chainer.reporter.report({'loss_X': self.lamdax*loss_X})        
        chainer.reporter.report({'loss1': loss1})        

        self.ip.cleargrads()
        loss_adv = F.softmax_cross_entropy(yp,label)
        loss2 = self.lamdap*(loss_adv)
        loss2.backward()
        ip_optimizer.update()
        chainer.reporter.report({'loss_adv': self.lamdap*loss_adv})        
        chainer.reporter.report({'loss_H': self.lamdap*loss_H})
        chainer.reporter.report({'loss2': loss2})        


In [7]:
import cv2
import matplotlib.pyplot as plt
import os



In [9]:
import chainer
from chainer import links as L
from chainer import functions as F
import numpy as np
import random
#import network
from network import D2AE, Identity

from chainer import training
from chainer.training import extension
from chainer.training import extensions
from iterator import MyIterator
from evaluation import eval_classification
from evaluation import eval_reconstruction
if __name__ == '__main__':

    config=Config()
    args=Args()
    chainer.cuda.get_device(config.gpu)
    #train_dataset, val_dataset, test_dataset, identityn = celebA_load(config)
    train_dataset, identityn=msceleb_loader(config.ms_celeb_dir)
    #print(len(train_dataset,))
    #train_dataset, val_dataset, test_dataset, identityn = celebA_rsa_loader('tmp')
    random.shuffle(train_dataset)
    #train_dataset=train_dataset[0:18*5+1]
    train_iter = MyIterator(config, train_dataset, args.batchsize)

    opts = {}
    #encoder=InceptionResNetV2()
    #serializers.load_npz(config.pretrainmodel_path, encoder)
    #d2ae=network.D2AE(encoder, identityn).to_gpu()
    #ip=network.Identity(identityn).to_gpu()
    d2ae=D2AE(config, encoder, identityn,config.gpu).to_gpu(config.gpu)
    copy_param(d2ae, encoder, identityn, config)
    ip=Identity(identityn).to_gpu(config.gpu)

    models = [d2ae,ip]
    updater_args = {
        "iterator": {'main': train_iter},
        "device": args.gpu
    }

    opts["opt_d2ae"] = make_sgd_optimizer(d2ae, args.lr)
    opts["opt_ip"] = make_sgd_optimizer(ip, args.lr)
    updater_args["optimizer"] = opts
    updater_args["models"] = models
    updater_args["config"] = config

    updater = Updater(**updater_args)
    report_keys = ["loss1","loss2","loss_I","loss_X","loss_adv", "loss_H"]

    trainer = training.Trainer(updater, (args.max_iter, 'iteration'), out=args.out)

    trainer.extend(MultistepShift('lr', 0.1, [args.epoch_iter*10,args.epoch_iter*20], 1e-2, optimizer=opts["opt_d2ae"]))
    trainer.extend(MultistepShift('lr', 0.1, [args.epoch_iter*10,args.epoch_iter*20], 1e-2, optimizer=opts["opt_ip"]))
    trainer.extend(extensions.LogReport(keys=report_keys,
                                        trigger=(args.display_interval, 'iteration')))
    trainer.extend(extensions.snapshot_object(
                d2ae, d2ae.__class__.__name__ + '_{.updater.iteration}.npz'), trigger=(args.snapshot_interval, 'iteration'))
    trainer.extend(extensions.PrintReport(report_keys), trigger=(args.display_interval, 'iteration'))
    trainer.extend(extensions.ProgressBar(update_interval=10))
    #trainer.extend(eval_classification(val_dataset, d2ae, ip, 1, 100, config.gpu,msg='val'), trigger=(args.evaluation_interval, 'iteration'),
    #           priority=extension.PRIORITY_WRITER)
    trainer.extend(eval_classification(train_dataset, d2ae, ip, 1, 10, config.gpu,msg='train'), trigger=(args.evaluation_interval, 'iteration'),
               priority=extension.PRIORITY_WRITER)
    trainer.extend(eval_reconstruction(train_dataset, d2ae, 10, config.gpu,msg='train', out=args.out), trigger=(args.evaluation_interval2, 'iteration'),
               priority=extension.PRIORITY_WRITER)





694 696


In [ ]:
    # Run the training
    %matplotlib inline
    trainer.run()


loss1       loss2       loss_I      loss_X      loss_adv    loss_H    
1.53539     1.05357     10.5143     1.53539     1.05357     1.05021     
     total [..................................................]  0.00%
this epoch [..................................................]  0.02%
        10 iter, 0 epoch / 1661926.0 iterations
       inf iters/sec. Estimated time to finish: 0:00:00.
1.00327     1.04856     10.5454     1.00327     1.04856     1.05649     
     total [..................................................]  0.00%
this epoch [..................................................]  0.04%
        20 iter, 0 epoch / 1661926.0 iterations
   0.89702 iters/sec. Estimated time to finish: 21 days, 10:38:22.635603.
0.911891    1.04792     10.4673     0.911891    1.04792     1.05757     
     total [..................................................]  0.00%
this epoch [..................................................]  0.05%
        30 iter, 0 epoch / 1661926.0 iterations
   0.9050

0.545701    1.04973     10.4479     0.545701    1.04973     1.05663     
     total [..................................................]  0.01%
this epoch [..................................................]  0.43%
       240 iter, 0 epoch / 1661926.0 iterations
   0.89626 iters/sec. Estimated time to finish: 21 days, 11:00:19.060799.
0.546186    1.05137     10.5785     0.546186    1.05137     1.05454     
     total [..................................................]  0.02%
this epoch [..................................................]  0.45%
       250 iter, 0 epoch / 1661926.0 iterations
   0.89647 iters/sec. Estimated time to finish: 21 days, 10:52:58.136908.
0.557547    1.04468     10.5414     0.557547    1.04468     1.06106     
     total [..................................................]  0.02%
this epoch [..................................................]  0.47%
       260 iter, 0 epoch / 1661926.0 iterations
   0.89718 iters/sec. Estimated time to finish: 21 days, 10:28:

0.463675    1.04221     10.4994     0.463675    1.04221     1.06231     
     total [..................................................]  0.03%
this epoch [..................................................]  0.85%
       470 iter, 0 epoch / 1661926.0 iterations
   0.89523 iters/sec. Estimated time to finish: 21 days, 11:31:34.745809.
0.485755    1.04418     10.4391     0.485755    1.04418     1.05982     
     total [..................................................]  0.03%
this epoch [..................................................]  0.87%
       480 iter, 0 epoch / 1661926.0 iterations
   0.89537 iters/sec. Estimated time to finish: 21 days, 11:26:42.701129.
0.474697    1.06096     10.4817     0.474697    1.06096     1.0433      
     total [..................................................]  0.03%
this epoch [..................................................]  0.88%
       490 iter, 0 epoch / 1661926.0 iterations
   0.89574 iters/sec. Estimated time to finish: 21 days, 11:13:

yt train
top10score:  0.0
top100score:  0.0
top1000score:  0.0
yp train
top10score:  0.0
top100score:  0.0
top1000score:  0.0
0.415563    1.05058     10.4799     0.415563    1.05058     1.05255     
     total [..................................................]  0.04%
this epoch [..................................................]  1.26%
       700 iter, 0 epoch / 1661926.0 iterations
   0.89592 iters/sec. Estimated time to finish: 21 days, 11:03:38.701309.
0.399117    1.05088     10.5166     0.399117    1.05088     1.05256     
     total [..................................................]  0.04%
this epoch [..................................................]  1.28%
       710 iter, 0 epoch / 1661926.0 iterations
   0.89616 iters/sec. Estimated time to finish: 21 days, 10:54:54.725161.
0.410925    1.0578      10.5745     0.410925    1.0578      1.04548     
     total [..................................................]  0.04%
this epoch [............................................

0.381278    1.05877     10.4586     0.381278    1.05877     1.0445      
     total [..................................................]  0.06%
this epoch [..................................................]  1.68%
       930 iter, 0 epoch / 1661926.0 iterations
   0.89361 iters/sec. Estimated time to finish: 21 days, 12:19:07.963138.
0.376012    1.05062     10.4533     0.376012    1.05062     1.05242     
     total [..................................................]  0.06%
this epoch [..................................................]  1.70%
       940 iter, 0 epoch / 1661926.0 iterations
   0.89352 iters/sec. Estimated time to finish: 21 days, 12:21:57.666816.
0.368238    1.04385     10.4934     0.368238    1.04385     1.0594      
     total [..................................................]  0.06%
this epoch [..................................................]  1.71%
       950 iter, 0 epoch / 1661926.0 iterations
   0.89347 iters/sec. Estimated time to finish: 21 days, 12:23: